In [14]:
import os
from io import BytesIO
from pathlib import Path
from multiprocessing import Pool

import pandas as pd

In [2]:
import sys
sys.path.append('../src')
from utils import run_cmd

In [4]:
def fn(filepath, outfile):
    cmd = f"conda run -n meningotype meningotype --finetype --porB --bast {filepath}"
    p = run_cmd(cmd)
    df = pd.read_csv(BytesIO(p.stdout), sep='\t')
    df.to_csv(outfile, sep='\t', index=False)

In [5]:
fn('/media/NGS/Sequence/Neisseria_meningitidis/Filter/GCA_000009105.1.fna',
   '/media/NGS/Sequence/Neisseria_meningitidis/meningotype/GCA_000009105.1.txt')

In [38]:
dirpath = Path('/media/NGS/Sequence/Neisseria_meningitidis/Filter')
outpath = Path('/media/NGS/Sequence/Neisseria_meningitidis/meningotype')

In [39]:
with Pool(32) as p:
    try:
        for filepath in dirpath.iterdir():
            outfile = outpath/(filepath.stem + '.txt')
            p.apply_async(fn, (filepath, outfile))
        p.close()
        p.join()
    except KeyboardInterrupt:
        p.terminate()

In [48]:
dirpath = Path('/media/NGS/SRA_1/NCBI_Neisseria_meningitidis_SRA/meningotype')

In [49]:
df = pd.concat(pd.read_csv(filepath, sep='\t') for filepath in dirpath.iterdir())
df.index = df['SAMPLE_ID'].apply(lambda x: Path(x).stem)
df = df.drop(['SAMPLE_ID', 'MLST'], axis=1)
df = df[df['SEROGROUP'] != 'ERROR: Check file exists and is in FASTA nucleotide format.']

In [50]:
df.to_csv('/media/NGS/SRA_1/NCBI_Neisseria_meningitidis_SRA/meningotype.tsv', sep='\t')